In [1]:
from utils.prep import split_sect
from utils.sub import submit
from utils.date import count_days
from approachs.naive import EachSection
from eval import smape

from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder


# prep

In [2]:
# load
tr_raw, te = pd.read_csv('data/train.csv'), pd.read_csv('data/test.csv')
target = 'price'


In [3]:
# prep before train-test split
## split sect
tr_raw[['section 1', 'section 2', 'section 3', 'section 4']] = tr_raw.plate.apply(split_sect).apply(pd.Series)
te[['section 1', 'section 2', 'section 3', 'section 4']] = te.plate.apply(split_sect).apply(pd.Series)

## date
tr_raw['days_old'] = count_days(tr_raw.date)
te['days_old'] = count_days(te.date)

#convert numeric strings
object_columns1 = tr_raw.select_dtypes(include=['object']).columns.tolist()
tr_raw[object_columns1] = tr_raw[object_columns1].apply(pd.to_numeric, errors='ignore') #better than astype as it decides whether int or float on its own
te[object_columns1] = te[object_columns1].apply(pd.to_numeric, errors='ignore') #better than astype as it decides whether int or float on its own


C:\Users\61668\AppData\Local\Temp\ipykernel_26328\2491354299.py:12: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  tr_raw[object_columns1] = tr_raw[object_columns1].apply(pd.to_numeric, errors='ignore') #better than astype as it decides whether int or float on its own
C:\Users\61668\AppData\Local\Temp\ipykernel_26328\2491354299.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  te[object_columns1] = te[object_columns1].apply(pd.to_numeric, errors='ignore') #better than astype as it decides whether int or float on its own


In [4]:
# eda
print(tr_raw['section 1'].nunique(),tr_raw['section 2'].nunique(),tr_raw['section 3'].nunique(),tr_raw['section 4'].nunique())

tr_raw[['section 2', 'section 4', 'days_old', 'price']].corr()

12 999 144 145


,section 2,section 4,days_old,price
section 2,1.000000,0.004341,-0.031774,0.025174
section 4,0.004341,1.000000,0.182044,-0.115608
days_old,-0.031774,0.182044,1.000000,0.047701
price,0.025174,-0.115608,0.047701,1.000000


In [5]:
# train test split
X_tr,  X_val, y_tr, y_val = train_test_split(tr_raw[[el for el in tr_raw.columns if el != target]], tr_raw[target], test_size=0.15, random_state=133)
print(len(X_tr),len(X_val),len(te))

43889 7746 7695


In [6]:
# prep after train-test split (dont want no leakage brooo)
## normalize (if non tree-based)
## nan impute
## etc


# model 

## naive

In [7]:
# section_method = EachSection()
# section_method.fit(X_tr, y_tr)
# te3=section_method.transform(te, ['section 2', 'section 4'])

# print(te3[['id','price']])

In [8]:
## naive
### based on section. mean section
section_method = EachSection()
section_method.fit(pd.concat([X_tr, X_val], axis=0), pd.concat([y_tr, y_val], axis=0))
temost=section_method.transform(te,method='most differing')

print(temost[['id','price']])

<class 'pandas.core.series.Series'> [332068.00350404 334202.85041762 335656.05162524 370623.59510302
 425489.24850895 427400.24617956 434855.43606557 435567.47391607
 454210.78223871 482692.67649537 530815.15738449 725602.96956443]


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
te4=te.copy()


In [16]:
submit(te4, f'just_mean')


test data valid and saved as just_mean.csv


In [ ]:
tr['section 1'].astype(float)

In [15]:
def one_hot(tr_ml1, fit_or_transform, object_columns2='all'):
    # one hot
    if object_columns2 == 'all':
        object_columns2 = tr_ml1.select_dtypes(include=['object']).columns.tolist()
    else:
        pass
    encoder = OneHotEncoder(sparse_output=False)
    if fit_or_transform == 'fit_transform':
        encoded = encoder.fit_transform(tr_ml1[object_columns2])
    elif fit_or_transform == 'transform':
        pass # for nowwww

    ## feature names for the encoded columns
    feature_names = []
    for i, col in enumerate(object_columns2):
        for category in encoder.categories_[i]:
            feature_names.append(f"{col}_{category}")

    ## DataFrame with encoded values
    encoded_df = pd.DataFrame(
        encoded,
        columns=feature_names,
        index=tr_ml1.index
    )
    return encoded_df

In [14]:
## ml
### first attempt at ml
'''
    cols: cats (section 1 and 3) and numerical (section 2 and 4 and days old)
    method: vanilla xgb 
    cats handling: one hot
'''

tr_ml1=tr[['section 1', 'section 2', 'section 3', 'section 4', 'days_old']]
te_ml1=te[['section 1', 'section 2', 'section 3', 'section 4', 'days_old']]

tr_ml1=one_hot(tr_ml1)

## combine it back
tr_ml1

,section 1_A,section 1_B,section 1_C,section 1_E,section 1_H,section 1_K,section 1_M,section 1_O,section 1_P,section 1_T,...,section 3_YC,section 3_YE,section 3_YH,section 3_YK,section 3_YM,section 3_YO,section 3_YP,section 3_YT,section 3_YX,section 3_YY
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51631,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51632,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51633,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
tr_ml1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51635 entries, 0 to 51634
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   section 1  51635 non-null  object
 1   section 2  51635 non-null  int64 
 2   section 3  51635 non-null  object
 3   section 4  51635 non-null  int64 
 4   days_old   51635 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 2.0+ MB


In [4]:
tr[['section 1', 'section 2', 'section 3', 'section 4']] = tr.plate.apply(split_sect).apply(pd.Series)

In [17]:
tr.groupby('section 2')['price'].mean().sort_values().head(10)


section 2
235    41111.111111
894    45285.571429
832    52000.000000
386    53125.000000
624    55357.142857
329    60000.000000
436    60000.000000
372    62000.000000
908    62200.000000
741    62555.555556
Name: price, dtype: float64

In [18]:
tr.groupby('section 2')['price'].mean().sort_values().tail(10)


section 2
704    1.202000e+06
093    1.274458e+06
895    1.287778e+06
001    1.402181e+06
859    1.426000e+06
039    1.525086e+06
614    1.555636e+06
777    1.983003e+06
026    2.157778e+06
293    6.042083e+06
Name: price, dtype: float64

In [25]:
dct.values()

dict_values([1, 2])

array([1, 2])

In [ ]:
dct = {'a':1, 'b':2}


1.5

In [19]:
tr.groupby('section 3')['price'].mean().sort_values().tail(10)


section 3
HH    7.627346e+05
OP    8.324309e+05
KK    8.370885e+05
EE    8.560062e+05
OO    8.752755e+05
YX    8.816390e+05
XX    9.501432e+05
AA    1.215905e+06
MM    1.266321e+06
MP    1.607407e+06
Name: price, dtype: float64

In [20]:
tr.groupby('section 3')['price'].mean().sort_values().head(10)


section 3
TM    177045.700000
TC    179345.501176
TX    182469.057935
TH    194615.745946
CP    216270.950877
HB    216321.962791
TE    219245.227642
KH    220067.114286
CX    221857.698347
TP    228194.697368
Name: price, dtype: float64

In [10]:
tr.groupby('section 4')['price'].mean().sort_values().head(20)


section 4
797     98940.443333
550    105706.478398
250    113322.192691
977    135218.769892
35     153382.352941
83     155000.000000
790    157614.550733
126    163159.128114
122    183935.064935
136    189836.065574
799    191702.606777
763    197202.380952
252    201032.258065
36     201688.679245
164    202666.666667
156    205409.836066
142    205604.830645
67     205849.379747
73     211911.764706
79     212227.272727
Name: price, dtype: float64

In [8]:
tr.groupby('section 4')['price'].mean().sort_values().tail(20)


section 4
777    7.173774e+05
61     7.275949e+05
87     7.277778e+05
05     8.017782e+05
90     8.123874e+05
55     8.232878e+05
97     9.977552e+05
42     1.017817e+06
99     1.172802e+06
01     1.178548e+06
77     1.345005e+06
63     1.452456e+06
173    1.500000e+06
39     1.555650e+06
66     1.842525e+06
24     2.132473e+06
121    2.266667e+06
17     2.528571e+06
08     3.076363e+06
88     5.500050e+07
Name: price, dtype: float64

In [41]:
def elements_not_in_train(train, test):
    missing_in_train_section = {}

    for i in tqdm(range(1,5)):
        missing_elements = []
        for item in tqdm(test[f'section {i}'].unique()):
            if item not in list(train[f'section {i}'].unique()):
                missing_elements.append(item)
        missing_in_train_section[i] = missing_elements
    
    print('test elements not in train:', missing_elements)
    return missing_elements

elements_not_in_train(tr, te)

100%|██████████| 4/4 [00:07<00:00,  1.76s/it]

test elements not in train: []


[]

KeyError: 'section 3'

In [3]:
tr 

,id,plate,date,price
0,1,X059CP797,2024-12-26 00:00:00,65000
1,2,Y800MH790,2024-07-12 21:31:37,100000
2,3,A212TX77,2024-04-18 00:00:00,290000
3,4,P001AY199,2025-01-03 00:27:15,680000
4,5,P001AY199,2025-01-10 09:32:41,750000
...,...,...,...,...
51630,51631,X023PP797,2025-01-29 00:00:00,70000
51631,51632,M004KA161,2025-01-31 00:00:00,1600000
51632,51633,E888EB199,2025-02-06 00:00:00,850000
51633,51634,X023XK77,2024-04-24 11:30:07,150000
